# Glaciers around the World: What influences glacial size?

### An interactive visualization and exploration by Eric Johnson

Our world contains many glaciers, ranging in size from over 11,000 square kilometeres to less than a fraction of a square kilometer. Why do glaciers come in such a wide variety of sizes? Glaciers form high in the mountains, and at sea level. They are found at the poles and in countries near the equator. They come in different types, forming in different places and because of different reasons. Do any of these factors correlate with glacier size? These visualizations will explore this question. 

These visualizations were created using data from different sources which can be found at the end of this article, with the main data set being used from the National Snow and Ice Data Center - the World Glacier Inventory, a catalog of over 130,000 glaciers that was created between 1900 and 2003.$^{1}$ This data serves as a representative sample of most glaciers on Earth in the end of the 20th century, with the exception of many Antarctican glaciers, as the data only includes glaciers north of -71 degrees South, which excludes most of the Antarctican mainland. The other main data set used is a collection of data on average rainfall for countries around the world, from worldbank.org.$^{2}$ 

In [1]:
import pandas as pd
import bqplot
import numpy as np
import traitlets
import ipywidgets
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
glaciers_file = 'world_glacier_inventory.zip'
rainfall_file = 'world_rainfall.zip'

glaciers = pd.read_csv(glaciers_file, encoding='ISO-8859-1')
rainfall = pd.read_csv(rainfall_file)

## Part 1: Glaciers by Area and Elevation

To start off our glacial exploration, let's consider the elevations where glaciers form. Glaciers form near the sea at both poles, where lengthy winters allow for vast expanses of ice. Glaciers also form in the mountains, where the temperatures can be significantly colder than at the base - for example, there are glaciers in places like South Africa and Venezuela! But is there a correlation between glacier size and elevation? Where do the largest glaciers form? Are they closer to sea level, or are the largest high up in the mountains? This first interactive visualization will explore any relationships between max elevation of glaciers and total area. 

In [3]:
glaciers = glaciers.dropna(subset=['total_area', 'max_elev', 'primary_class'])
glaciers = glaciers[glaciers.total_area != 0]
glaciers = glaciers[(glaciers.lat != 0) & 
                    (glaciers.lon != 0)]

In [4]:
nlong = 50
nlat = 50

long_bins = np.linspace(-180, 180, nlong+1)
lat_bins = np.linspace(-90, 90, nlat+1)

hist2d, long_edges, lat_edges = np.histogram2d(glaciers['lon'], glaciers['lat'],
                                                 weights=glaciers['max_elev'], 
                                                 bins=[long_bins, lat_bins], 
                                                 )

long_centers = [(long_edges[i] + long_edges[i+1]) * 0.5 for i in range(len(long_edges)-1)]
lat_centers = [(lat_edges[i] + lat_edges[i+1]) * 0.5 for i in range(len(lat_edges)-1)]

np.log10(hist2d)

hist2d[hist2d <=0] = np.nan
hist2d = np.log10(hist2d)
hist2d = hist2d.T

hist2d

C:\Users\Eric\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in log10
  from ipykernel import kernelapp as app


array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [5]:
col_sc = bqplot.ColorScale(scheme='GnBu')
x_sc = bqplot.LinearScale()
y_sc = bqplot.LinearScale()

c_ax = bqplot.ColorAxis(scale=col_sc, orientation='vertical', side='right', label='log(m)')
x_ax = bqplot.Axis(scale=x_sc, label='Longitude')
y_ax = bqplot.Axis(scale=y_sc, label='Latitude', orientation='vertical')

heat_map = bqplot.GridHeatMap(color=hist2d, row=lat_centers, column=long_centers,
                             scales={'color':col_sc, 'row':y_sc, 'column':x_sc},
                             interactions={'click':'select'},
                             anchor_style={'fill': 'red'},
                             selected_style={'opacity':1.0},
                             unselected_style={'opacity':1.0})

### Elevation and Area Plot ###

x_scl = bqplot.LinearScale()
y_scl = bqplot.LinearScale()
ax_xcl = bqplot.Axis(label='Max Elevation in m', scale=x_scl)
ax_ycl = bqplot.Axis(label='Total Area in sq.m', scale=y_scl, 
                     orientation='vertical', side='left')


i, j = 0, 0 
longs = [long_edges[j], long_edges[j+1]]
lats = [lat_edges[i], lat_edges[i+1]]
region_mask = ((glaciers['lat'] >= lats[0]) & (glaciers['lat'] < lats[1]) &\
              (glaciers['lon'] >= longs[0]) & (glaciers['lon'] < longs[1]))

elev, elev_edges = np.histogram(glaciers['max_elev'][region_mask], 
                                weights=glaciers['total_area'][region_mask], 
                                bins=10)

elev_centers = [(elev_edges[i] + elev_edges[i+1]) * 0.5 for i in range(len(elev_edges)-1)]

elev_hist = bqplot.Lines(x=elev_centers, y=elev,
                       scales={'x':x_scl, 'y':y_scl})

fig_elev = bqplot.Figure(marks=[elev_hist], axes=[ax_xcl, ax_ycl])

### Tracking Changes ###

mySelectedLabel = ipywidgets.Label()
def get_data_value(change):
    i, j = change['owner'].selected[0]
    v = hist2d[i, j]
    mySelectedLabel.value = 'Max Elevation in log(m) = ' + str(v)
    longs = [long_edges[j], long_edges[j+1]]
    lats = [lat_edges[i], lat_edges[i+1]]
    region_mask = ((glaciers['lat'] >= lats[0]) & (glaciers['lat'] < lats[1]) &\
              (glaciers['lon'] >= longs[0]) & (glaciers['lon'] < longs[1]))
    
    elev, elev_edges = np.histogram(glaciers['max_elev'][region_mask], 
                                weights=glaciers['total_area'][region_mask], 
                                bins=10)
    elev_centers = [(elev_edges[i] + elev_edges[i+1]) * 0.5 for i in range(len(elev_edges)-1)]
    elev_hist.x = elev_centers
    elev_hist.y = elev
    
heat_map.observe(get_data_value, 'selected')

fig = bqplot.Figure(marks=[heat_map], axes=[c_ax, y_ax, x_ax], 
                    title='World Glaciers by Max Elevation & Area')

fig_elev.layout.max_width = '600px'
fig_elev.layout.max_height = '500px'
fig.layout.min_width = '800px'

ipywidgets.VBox([mySelectedLabel, ipywidgets.HBox([fig_elev]),
                fig])

A Jupyter Widget

This visualization is a heat map, where each square represents the relative maximum elevation of glaciers in that square. The darker the color, the higher the elevation for the glaciers in that square. Each square can be clicked on, and will update the line graph above the heat map. The line graph compares the max elevation for each glacier in the selected square and shows the total area for the glaciers at each elevation. This is an easy way to compare different glaciers around the world and see at which elevations the largest glaciers form.

Playing around with this visualization will help to show that glaciers exist at many different elevations and come in many different sizes. 

## Part 2: Glaciers by Type: What kinds of glaciers exist on Earth? Why should we care?

In this section, we'll take a look at the different types of glaciers. It is important to get a closer look at the different kinds of glaciers, because there are many different factors that come into play when considering why some glaciers are larger than others. First we looked at elevation, to see if that had any correlation on size. Now we can look at glacier type, and see if there is any connection between the type of glacier and its size. 

Before we get started with this section, I'll briefly mention the ten different classifications of glacier types used by the NSIDC: Continental Ice Sheet, Ice Field, Ice Cap, Outlet Glacier, Valley Glacier, Mountain Glacier, Glacieret and Snowfield, Ice Shelf, Rock Glacier, and Miscellaneous. These types might not mean much right now, but with a little investigation, we can easily discover what the differences are between them. Continental Ice Sheet, for example, is a glacier that covers areas of continental land masses, such as the glaciers that cover Greenland.$^{3}$ An Ice Shelf is a glacier that floats on the sea and is connected to another glacier on the coast - the most famous example of this is the Ross Ice Shelf in Antarctica.$^{4}$ And a Valley Glacier is a glacier that has formed in a valley.$^{5}$ The other glacier types can be easily looked up for more detailed information, and the User Guide for this data set provides a brief explanation of each type as well, which can be found in the sources at the end of this article.$^{1}$

In [6]:
class_name = ['Miscellaneous', 
             'Continental Ice Sheet',
             'Ice Field', 
             'Ice Cap', 
             'Outlet Glacier', 
             'Valley Glacier', 
             'Mountain Glacier', 
             'Glacieret and Snowfield', 
             'Ice Shelf', 
             'Rock Glacier']

glaciers_class = glaciers.groupby('primary_class')['total_area'].mean()
glaciers_class = glaciers_class.reset_index()
glaciers_class['primary_class_name'] = class_name

glaciers_area = glaciers.groupby('primary_class')['total_area'].sum()
glaciers_area = glaciers_area.reset_index()
glaciers_area['primary_class_name'] = class_name

In [7]:
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.LinearScale()

tt = bqplot.Tooltip(fields=['x', 'y'], labels=['Primary Class', 'Average Total Area'])

bars = bqplot.Scatter(x=glaciers_class['primary_class_name'], y=glaciers_class['total_area'], 
                  scales={'x': x_sc, 'y': y_sc}, tooltip=tt)

x_ax = bqplot.Axis(scale=x_sc, label='Primary Class Type')
y_ax = bqplot.Axis(scale=y_sc, orientation='vertical', label='Average Total Area in Square Meters')

fig = bqplot.Figure(marks=[bars], axes=[x_ax, y_ax], title='Primary Class Type by Average Total Area')
fig

A Jupyter Widget

For all of the visualizations in this section, you can hover over a point to see the specific glacier type as well as the exact area value. 

The above visualization looks at the average area for a glacier of each type. It is easy to see that Continental Ice Sheets, on average, are far larger than any other type of glacier. Most of the other types are fairly close in average area to each other, and in fact, most of these are so close that it is hard to see which types are larger or smaller than the others. To get a better look at this data, the plot can be redrawn using a log scale, which essentially will exaggerate any differences near the bottom of the scale. 

In [8]:
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.LogScale()

tt = bqplot.Tooltip(fields=['x', 'y'], labels=['Primary Class', 'Average Total Area'])

bars = bqplot.Scatter(x=glaciers_class['primary_class_name'], y=glaciers_class['total_area'], 
                  scales={'x': x_sc, 'y': y_sc}, tooltip=tt)

x_ax = bqplot.Axis(scale=x_sc, label='Primary Class Type')
y_ax = bqplot.Axis(scale=y_sc, orientation='vertical', label='Average Total Area in Square Meters as a Log Scale')

fig = bqplot.Figure(marks=[bars], axes=[x_ax, y_ax], title='Primary Class Type by Average Total Area as a Log Scale')
fig

A Jupyter Widget

The plot above is the exact same data as the previous plot, just plotted on the log scale. As you can see, the differences between all of the glacier types near the bottom are greatly exaggerated, so now it is easy to see that Ice Shelf glaciers have, on average, the smallest area. Note that by hovering over the data points, the average area number is the same as in the previous graph! It is simply a difference in how it is plotted. 

In [9]:
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.LinearScale()

tt = bqplot.Tooltip(fields=['x', 'y'], labels=['Primary Class', 'Total Area'])

bars = bqplot.Scatter(x=glaciers_area['primary_class_name'], y=glaciers_area['total_area'], 
                  scales={'x': x_sc, 'y': y_sc}, tooltip=tt)

x_ax = bqplot.Axis(scale=x_sc, label='Primary Class Type')
y_ax = bqplot.Axis(scale=y_sc, orientation='vertical', label='Total Area in Square Meters')

fig = bqplot.Figure(marks=[bars], axes=[x_ax, y_ax], title='Primary Class Type by Total Area')
fig

A Jupyter Widget

Finally, the plot above shows the glacier type by total area - that is, the area of all glaciers of that kind added together, rather than the average area for a glacier of that type. This graph shows us how much of the world's glaciers are which type of glacier. As expected, there are more Continental Ice Sheets than any other glacier type on Earth. Interestingly, this plot looks very similar to the log scale plot of average glacier area we looked at previously - reassuring us that glacier type is a good indicator of relatively how large or small a glacier will be - seemingly regardless of other factors. 

From this section, we can see that Continental Ice Sheets are always far larger than other glacier types. We can take this into consideration when looking at the other factors for what accounts for differences in glacial size. 

## Part 3: Glaciers by Country: Rainfall, Elevation, and Area

For part 3, we'll take a look at an additional factor - rainfall. Does rainfall in an area influence glacier size? This rainfall data for each country was gathered from worldbank.org.$^{2}$ The visualizations here will also compare elevation relative to rainfall, to see if there are any connections between elevation and rainfall and glacier area. Rainfall could potentially have a correlation with glacier size because precipitation falling in a cold area could lead to snow or ice, which could be an indicator of locations where glaciers are more likely to form, and where they do form, are more likely to be larger in size. Elevation is included in these comparisons because in some places, elevation does have a correlation with rainfall, due to cooling of air as it rises over mountains. Take a look at the following visualizations to see if rainfall and elevation have a correlation with glacier size!

In [10]:
country_dict = {'AF':'Afghanistan', 'AQ':'Antarctica', 'AR':'Argentina', 'AT':'Austria',
               'BO':'Bolivia', 'BT':'Bhutan', 'CA':'Canada', 'CH':'Switzerland', 'CL':'Chile',
               'CN':'China', 'CO':'Colombia', 'DE':'Germany', 'EC':'Ecuador', 'ES':'Spain',
               'FR':'France', 'GL':'Greenland', 'GS':'South Georgia', 
               'HM':'Heard Island and McDonald Islands', 'ID':'Indonesia', 'IN':'India', 'IS':'Iceland',
               'IT':'Italy', 'KE':'Kenya', 'MX':'Mexico', 'NO':'Norway', 'NP':'Nepal',
               'NZ':'New Zealand', 'PE':'Peru', 'PK':'Pakistan', 'SE':'Sweden', 
                'SU':'Russian Federation', 'TF':'French Southern Territories', 'TZ':'Tanzania', 
                'UG':'Uganda', 'US':'United States', 'VE':'Venezuela, RB', 'ZA':'South Africa', 
                'ZR':'Congo, Dem. Rep.'}

for i in country_dict:
    glaciers.loc[glaciers['political_unit'] == i, 'political_unit'] = country_dict[i]
    
header = rainfall.iloc[3]
rainfall = rainfall[4:]
rainfall.columns = header

glaciers_by_country = glaciers.groupby('political_unit')['total_area', 'max_elev', 'lat', 'lon'].mean()
glaciers_by_country = glaciers_by_country.reset_index()

rainfall['avg'] = rainfall.mean(axis=1)
rainfall = rainfall[['Country Name', 'Country Code', 'avg']]
rainfall = rainfall.set_index('Country Name')
rain_glaciers = glaciers_by_country.join(rainfall, on='political_unit')
rain_glaciers = rain_glaciers.dropna()

In [15]:
### Rainfall Plot ###
x_sc = bqplot.LinearScale()
y_sc = bqplot.LinearScale()
c_sc = bqplot.ColorScale(scheme='RdYlGn')

x_ax = bqplot.Axis(scale=x_sc, label='Longitude')
y_ax = bqplot.Axis(scale=y_sc, label='Latitude', orientation='vertical')
c_ax = bqplot.ColorAxis(scale=c_sc, label='Rain (mm)', orientation='vertical', side='right')

scatters = bqplot.Scatter(x=rain_glaciers['lon'], y=rain_glaciers['lat'],
                         color=rain_glaciers['avg'],
                         scales={'x':x_sc, 'y':y_sc, 'color':c_sc})

fig = bqplot.Figure(marks=[scatters], axes=[x_ax, y_ax, c_ax], title='Average Rainfall by Country')

### Elevation Plot ###
x_sc2 = bqplot.LinearScale()
y_sc2 = bqplot.LinearScale()
c_sc2 = bqplot.ColorScale(scheme='RdYlGn')

x_ax2 = bqplot.Axis(scale=x_sc2, label='Longitude')
y_ax2 = bqplot.Axis(scale=y_sc2, label='Latitude', orientation='vertical')
c_ax2 = bqplot.ColorAxis(scale=c_sc2, label='Elev', orientation='vertical', side='right')

scatters2 = bqplot.Scatter(x=rain_glaciers['lon'], y=rain_glaciers['lat'],
                           color=rain_glaciers['max_elev'],
                           scales={'x':x_sc2, 'y':y_sc2, 'color':c_sc2})

fig2 = bqplot.Figure(marks=[scatters2], axes=[x_ax2, y_ax2, c_ax2], title='Average Max Glacier Elevation by Country')

### Area Plot ###

x_sc3 = bqplot.LinearScale()
y_sc3 = bqplot.LinearScale()
c_sc3 = bqplot.ColorScale(scheme='RdYlGn')

x_ax3 = bqplot.Axis(scale=x_sc3, label='Longitude')
y_ax3 = bqplot.Axis(scale=y_sc3, label='Latitude', orientation='vertical')
c_ax3 = bqplot.ColorAxis(scale=c_sc3, label='Area (sq.m)', orientation='vertical', side='right')

scatters3 = bqplot.Scatter(x=rain_glaciers['lon'], y=rain_glaciers['lat'],
                           color=rain_glaciers['total_area'],
                           scales={'x':x_sc3, 'y':y_sc3, 'color':c_sc3})

fig3 = bqplot.Figure(marks=[scatters3], axes=[x_ax3, y_ax3, c_ax3], title='Average Glacier Area by Country')

fig2.layout.max_width = '800px'
fig2.layout.max_height = '400px'
fig3.layout.max_width = '800px'
fig3.layout.max_height = '400px'

ipywidgets.VBox([ipywidgets.HBox([fig2, fig3]), fig])

A Jupyter Widget

Taking a look at the three plots above, we see average glacier elevation and average glacier area in the top row, and average rainfall at the bottom. These plots were made by averaging all information about glaciers by country, rather than longitude and latitude segments as before, due to the nature of the rainfall data. Therefore, each data point on these plots represents all of the data for a single country. The locations of the points were chosen to represent the average of the latitudes and longitudes of all glaciers in that country (so, for example, the points that represent the United States glaciers are up where Alaska is, since that's where the majority of glaciers in the United States exist, skewing the average latitude and longitudes up north). 

It seems there is not much of a correlation between average rainfall and glacier size. There is somewhat of a correlation between rainfall and elevation, although only in some areas (the Himalayas/Tibetan Plateau are one major exception). Even though there isn't a correlation here, that's okay, because we've eliminated another variable as to why some glaciers are larger than others - we know that average rainfall doesn't seem to have any effect on glacier size. 

So, to summarize these groups of visualizations, there are many potential reasons why glaciers vary in size. We've explored a couple of those hypotheses here, including elevation, glacier type, and average rainfall. And hopefully you've learned more about glaciers in the process! 

** Sources:
1. WGMS, and National Snow and Ice Data Center (comps.). 1999 to present, updated 2012. World Glacier Inventory, Version 1. Boulder, Colorado USA. NSIDC: National Snow and Ice Data Center. doi: https://doi.org/10.7265/N5/NSIDC-WGI-2012-02. March 31, 2019.
2. https://data.worldbank.org/indicator/AG.LND.PRCP.MM?type=shaded&view=map
3. https://www.nationalgeographic.org/encyclopedia/ice-sheet/
4. https://nsidc.org/cryosphere/quickfacts/iceshelves.html
5. https://nsidc.org/cryosphere/glaciers/questions/types.html